In [ ]:
pip install pydicom nibabel numpy torch torchvision segmentation-models-pytorch scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install albumentations

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from scipy.ndimage import zoom
from operator import itemgetter
from sklearn.metrics import jaccard_score


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
num_classes = 7
batch_size = 8
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/segmentation_with_racegender.csv"
test_augmentations = A.Compose([
    A.Resize(height=256, width=256),
    A.Normalize(mean=(0.485,), std=(0.229,)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, image_files, mask_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.img_files = image_files
        self.mask_files = mask_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        dicom_image = pydicom.dcmread(os.path.join(self.img_root, self.img_files[int(float(idx))]))
        image = dicom_image.pixel_array.astype(np.float32)
        annotation = nib.load(os.path.join(self.mask_root, self.mask_files[int(float(idx))]))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)

        if annotation_data.ndim > 2 and annotation_data.shape[-1] != 1:
            raise ValueError('Mask has multiple channels')

        if image.shape != annotation_data.shape:
            zoom_factors = np.array(image.shape) / np.array(annotation_data.shape)
            annotation_data = zoom(annotation_data, zoom_factors, order=0)

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(self.img_files[int(float(idx))].split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))
paired_files = list(zip(image_files, mask_files))
random.shuffle(paired_files)

train_size = int(0.2 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]


train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, [p[0] for p in train_pairs], [p[1] for p in train_pairs], num_classes,
    transforms=test_augmentations
)


model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights="imagenet",
    in_channels=1,
    classes=num_classes,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()


valid_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, [p[0] for p in valid_pairs], [p[1] for p in valid_pairs], num_classes,
    transforms=test_augmentations
)

valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)


num_epochs = 100
increment_percentage = 0.1

for epoch in range(num_epochs):
    if epoch == 0:
        train_size = int(0.2 * len(paired_files))
    else:
        train_size += int(increment_percentage * len(paired_files))
        if train_size > len(paired_files):
            train_size = len(paired_files)
    train_pairs = paired_files[:train_size]

    train_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, [p[0] for p in train_pairs], [p[1] for p in train_pairs], num_classes,
        transforms=test_augmentations
    )

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)


    model.train()

    iou_racegender_list = []

    for batch_idx, (images, masks, racegender) in enumerate(train_loader):
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)
        loss = criterion(outputs, masks.argmax(dim=1))
        loss.backward()
        optimizer.step()


        train_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )
        iou_racegender_list.append((train_iou, batch_idx, racegender))

    if epoch > 2:
        # Sort the list of IoU values, indices, and racegender in descending order based on IoU values
        iou_racegender_list.sort(reverse=True, key=itemgetter(0))

        # Extract unique racegenders
        racegenders = set(racegender for _, _, racegender in iou_racegender_list)

        # Create dictionaries to store sorted lists of IoU values and indices for each racegender
        racegender_lists = {racegender: [] for racegender in racegenders}
        for iou, idx, racegender in iou_racegender_list:
            racegender_lists[racegender]: [] = iou, idx

        # Reconstruct DataLoader based on sorted indices and alternating racegenders
        sorted_indices = []
        for racegender in racegenders:
            sorted_indices.extend(racegender_lists[racegender])

        train_loader = DataLoader(
            train_set,
            batch_size=batch_size,
            sampler=torch.utils.data.sampler.SubsetRandomSampler(sorted_indices),
            num_workers=2
        )

    model.eval()
    valid_iou_list = []

    for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        valid_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )

        valid_iou_list.append(valid_iou)

    valid_iou_avg = np.mean(valid_iou_list)

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou:.4f} - Validation IoU: {valid_iou_avg:.4f}")


/tmp/ipykernel_259462/3212025261.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_259462/3212025261.py:80: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_259462/3212025261.py:80: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empt

Epoch [1/100] - Train IoU: 0.0880 - Validation IoU: 0.0633


/tmp/ipykernel_259462/3212025261.py:80: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_259462/3212025261.py:80: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [2/100] - Train IoU: 0.1321 - Validation IoU: 0.1257


/tmp/ipykernel_259462/3212025261.py:80: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_259462/3212025261.py:80: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


...

Epoch [98/100] - Train IoU: 0.9587 - Validation IoU: 0.9629


/tmp/ipykernel_259462/3212025261.py:80: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_259462/3212025261.py:80: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [99/100] - Train IoU: 0.9512 - Validation IoU: 0.9591


/tmp/ipykernel_259462/3212025261.py:80: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_259462/3212025261.py:80: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [100/100] - Train IoU: 0.9521 - Validation IoU: 0.9634


In [ ]:
saved_model_path = 'trained_model_ Multi Diversity 100 epoch with Decay1e-5 ioulib .pth'


torch.save(model.state_dict(), saved_model_path)


print(f"Model saved to {saved_model_path}")

Model saved to trained_model_ Multi Diversity 100 epoch with Decay1e-5 ioulib .pth
